*GPUs. Процесс тренировки и overfitting/underfitting на практике,. Learning rate annealing. Batch Normalization. Ансамбли. Что нового в 2018.*

---

(последняя из лекций про нейросети в целом; далее речь будет идти только о конкретных областях и реализациях применения)

Если что-то непонятно (здесь или далее), ОЧЕНЬ ВАЖНО **вернуться к (по факту, самым необходимым для понимания) лекциям 1–4 и разобраться**!!! Да-да!

## Почему обучение на видеокартах (GPU)?

*(тут мне и так всё понятно, расписывать ничего не хочу и не буду)*

## The Secret Ingredient is... Training Process

![alt text](attach1.png)

Ключом к успешному обучение нейросети является чёткое понимание процесса тренировки — куда всё движется и что делать, чтобы перейти от underfit/overfit к optimal.

*Для Torch есть полезный инстумент для анализа метрик **Tensorboard**.*

Потренируемся по графикам (некоторой метрики точности у val и train) определять, в каком "режиме" находится модель, и что с этим делать:

#### Ситуация №1

Train loss и Val loss уменьшаются медленно и идут близко друг к другу — это overfit, underfit или всё OK? **ОТВЕТ ПОД КАРТИНКОЙ**

![alt text](attach2.png)

Это **underfitting** — модель недостаточно сложная или процесс тренировки слишком слабый.

#### Что можно сделать?

- Из прошлых лекций:
    - Взять модель помощнее
    - Изменить подход к данным
    - Использовать Preprocessing, если он ещё не используется
    - Использовать хороший метод тренировки (например, Adam вместо обычного градиентного спуска) 
- Из нового:
    - (довольно очевидное) Попробовать подобрать (экспериментальным путём — а как иначе?) Learning Rate
    - (менее очевидное) **Learning Rate Annealing (Decay)**:

        ![alt text](attach3.png)

#### Ситуация №2

Модель тренируется довольно хорошо — train продолжает падать и падать... но val через какое-то время перестаёт уменьшаться, и даже начинает увеличиваться! Что это такое? **ОТВЕТ ПОД КАРТИНКОЙ**

![alt text](attach4.png)

Это типичный случай **overfitting** — модель, по сути, тупо запомнила тренировочные данные, а не их конкретные признаки, и начала терять способность распознавать реальные данные.

#### Что можно сделать?

- Из прошлых лекций:
    - **Регулязиарция** (например, уже известная нам **L2-регуляризация**, или **Dropout** (обычно 1–2 раза в конце сети)):

        ![alt text](attach5.png)

## Batch Normalization

#### Есть ли универсальные методы И для overfit, И для underfit?

 Да, и BN — самый известный из такиз методов. Он работает с проблемой "взрывающихся (улетающих высоко вверх) и затухающих (улетающих в ноль) нейронов", добиваясь ситуации, чтобы "сила" активаций нейронов была примерно одного масштаба.

![alt text](attach6.png)

Принцип работы — batchnorm линейно масштабирует (умножит на что-нибудь и что-нибудь прибавит) выход слоя после взвешивания так, чтобы у весов было предсказуемое среднее и стандартное отклоненени.

![alt text](attach7.png)

![alt text](attach8.png)

## Поиск гиперпараметров

#### Что мы имеем в виде под гиперпараметрами?

- Начальный Learning Rate
- Коэффициент Learning Rate Annealing
- Коэффициент L2-регуляризации
- Размер batch
- ...

#### TLDR — методом тыка — сначала просто наугад, потом "по чуйке".

Общие рекомендации для этого процесса:
1. Сперва ищем в широком диапозоне, пока не находим +- оптимальный вариант. А после этого начинаем немного от него отходить в одну-другую сторону, где лучше результат.
2. Log space:
    - `lr = random(10, 10000)` — ПЛОХО
    - `lr = 10**random(1, 4)` — ХОРОШО

---

## **Короче! Начинать с...**
1. **Preprocessing** — Вычитаем среднее из данных
2. Оптимизатор — **Adam**
3. **Batch Normalization** — добро
4. **Learning Rate Annealing на плато**
5. Перебираем гиперпараметры; важнее всего **learning rate**
6. **СМОТРИМ НА ГРАФИКИ ТРЕНИРОВКИ!**

---

А для получения дополнительных % точностей уже ближе к концу процесса создания модели можно использовать вот эти трюки:

## Ансамбль моделей

*(к музыкальным ансамблям отношения практически никакого не имеет, just in case)*

![alt text](attach9.png)

Если натренировать не одну модель, а несколько, желательно поразнообразнее, но каждая имеет хорошую точность, а затем их предсказания усреднить, то итоговая точность будет выше на 1–5%!

#### Откуда брать разнообразные модели?
- **Использовать разные модели и подходы**
- **Cross-validation folds** (усреднение одной и той же модели, натренированной на разных trans/val/test-разбиениях датасета)
- ***Креатив!***

## BACK TO THE "PRESENT"

*Свежие (ну, почти) подходы из 2018–2019 гг.*

#### LR Range Test + Cyclical Learning Rate

Отдельный экспериментаьный запуск на train исключительно для подбора оптимального диапазона LR — по ходу обучения через $N$ эпох Learning Rate постепенно повышается от околонулевого до очень большого, а затем анализируем график точности.

Оттуда мы извлкаем минимальный и максимальный Learning Rate.

Затем, при уже нормальной тренировке модели, мы циклически повышаем-понижаем LR от минимально-оптиматльного до макисмально-оптимального, в надежде, что это поможет нейронке "выбраться" из локального минимума и найти точку более низкую, тем самым получить точность предсказания повыше.

На практике, это почти всегда помогает прийти либо к более, либо хотя бы к не менее точной точности модели.

![alt text](attach10.png)

#### Инженерный подход к перебору гиперпараметров

![alt text](attach11.png)

## (для общего развития) Что и как используют в проде?

![alt text](attach12.png)

На этом всё! Дальше лекции будут более конкретными по отедльным темам.